## Explore Squad v1.1

In [112]:
import json

In [27]:
import os, sys
path_project = '~/fontes/exqa-complearning/'

sys.path.append(os.path.abspath('.'))
sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath(r'../..'))
print(sys.path)

['/home/borela/fontes/exqa-complearning/source/exploration', '/home/borela/miniconda3/envs/relevar-busca/lib/python37.zip', '/home/borela/miniconda3/envs/relevar-busca/lib/python3.7', '/home/borela/miniconda3/envs/relevar-busca/lib/python3.7/lib-dynload', '', '/home/borela/miniconda3/envs/relevar-busca/lib/python3.7/site-packages', '/home/borela/miniconda3/envs/relevar-busca/lib/python3.7/site-packages/IPython/extensions', '/home/borela/.ipython', '/home/borela/fontes/exqa-complearning/source/exploration', '/home/borela/fontes/exqa-complearning/source/exploration/.\\.', '/home/borela/fontes/exqa-complearning/source/exploration', '/home/borela/fontes/exqa-complearning/source/exploration', '/home/borela/fontes/exqa-complearning/source/exploration', '/home/borela/fontes/exqa-complearning/source/exploration', '/home/borela/fontes/exqa-complearning/source/exploration', '/home/borela/fontes/exqa-complearning/source/exploration/.\\.', '/home/borela/fontes/exqa-complearning/source/exploration'

In [23]:
%cd ..


/home/borela/fontes/exqa-complearning


In [28]:
%pwd

'/home/borela/fontes/exqa-complearning'

In [30]:
from transformers.data.processors.squad import SquadV1Processor # SquadV2Processor

processor = SquadV1Processor()
examples = processor.get_dev_examples("data/dataset/squad/", filename="dev-v1.1.json")
print(len(examples))

100%|██████████| 48/48 [00:02<00:00, 23.14it/s]

10570


In [ ]:
processor = SquadV1Processor()
examples = processor.get_dev_examples("data/dataset/squad/", filename="dev-v1.1.json")
print(len(examples))

In [31]:
def display_example(qid):    
    from pprint import pprint

    idx = qid_to_example_index[qid]
    q = examples[idx].question_text
    c = examples[idx].context_text
    a = [answer['text'] for answer in examples[idx].answers]
    
    print(f'Example {idx} of {len(examples)}\n---------------------')
    print(f"Q: {q}\n")
    print("Context:")
    pprint(c)
    print(f"\nTrue Answers:\n{a}")



In [32]:
    
# generate some maps to help us identify examples of interest
qid_to_example_index = {example.qas_id: i for i, example in enumerate(examples)}
qid_to_has_answer = {example.qas_id: bool(example.answers) for example in examples}
answer_qids = [qas_id for qas_id, has_answer in qid_to_has_answer.items() if has_answer]
# no_answer_qids = [qas_id for qas_id, has_answer in qid_to_has_answer.items() if not has_answer]


In [33]:
display_example(answer_qids[1300])


Example 1300 of 10570
---------------------
Q: Where was the  Continental Edison Company located?

Context:
('In 1882, Tesla began working for the Continental Edison Company in France, '
 'designing and making improvements to electrical equipment. In June 1884, he '
 'relocated to New York City:57–60 where he was hired by Thomas Edison to work '
 "at his Edison Machine Works on Manhattan's lower east side. Tesla's work for "
 'Edison began with simple electrical engineering and quickly progressed to '
 'solving more difficult problems.')

True Answers:
['France', 'in France', 'France']


In [143]:
answer_qids[1]

'56be4db0acb8001400a502ed'

In [43]:
answer_qids[1300], type(answer_qids[1300])

('56e0d54a7aa994140058e76a', str)

In [44]:
qid = answer_qids[1300]

In [45]:
idx = qid_to_example_index[qid]
q = examples[idx].question_text
c = examples[idx].context_text
a = [answer['text'] for answer in examples[idx].answers]
q, c


('Where was the  Continental Edison Company located?',
 "In 1882, Tesla began working for the Continental Edison Company in France, designing and making improvements to electrical equipment. In June 1884, he relocated to New York City:57–60 where he was hired by Thomas Edison to work at his Edison Machine Works on Manhattan's lower east side. Tesla's work for Edison began with simple electrical engineering and quickly progressed to solving more difficult problems.")

In [36]:
import copy


In [144]:
from source.fine_tuning.responder_extracao import responder_extracao, json_exemplo  # pylint: disable=wrong-import-position # precisa dos sys.path antes
from source.fine_tuning.modelo_reader import lista_modelos_reader

In [38]:
json_fine_tuning = copy.deepcopy(json_exemplo)
json_fine_tuning["tamanho_max_resposta"] = 30
json_fine_tuning['texto_pergunta'] = q
json_fine_tuning['texto_contexto'] = c

In [49]:
json_fine_tuning

{'texto_pergunta': 'Where was the  Continental Edison Company located?',
 'top_k_reader': 3,
 'tamanho_max_resposta': 30,
 'texto_contexto': "In 1882, Tesla began working for the Continental Edison Company in France, designing and making improvements to electrical equipment. In June 1884, he relocated to New York City:57–60 where he was hired by Thomas Edison to work at his Edison Machine Works on Manhattan's lower east side. Tesla's work for Edison began with simple electrical engineering and quickly progressed to solving more difficult problems.",
 'sigla_modelo_reader': 'pierreguillou/bert-large-cased-squad-v1.1-portuguese'}

In [48]:
resposta = responder_extracao(json_fine_tuning)
print(f"resposta {resposta}")


resposta {'lista_sugestao_resposta': [{'texto_resposta': 'New York City:57–60', 'score': 0.9129816889762878, 'lista_referencia': [[164, 183]]}, {'texto_resposta': 'New York City:57–60 where he was hired by Thomas Edison to work at his Edison Machine Works', 'score': 0.0031480903271585703, 'lista_referencia': [[164, 255]]}]}


In [51]:
json_fine_tuning['texto_pergunta']  = "Em 1882, Tesla começou a trabalhar para a Continental Edison Company na França, projetando e fazendo melhorias em equipamentos elétricos. Em junho de 1884, ele se mudou para Nova York: 57-60, onde foi contratado por Thomas Edison para trabalhar em sua Edison Machine Works, no lado leste inferior de Manhattan. O trabalho de Tesla para Edison começou com engenharia elétrica simples e rapidamente progrediu para resolver problemas mais difíceis."
json_fine_tuning['texto_contexto'] = "Onde estava localizada a Continental Edison Company?"

In [50]:
a

['France', 'in France', 'France']

In [52]:
resposta = responder_extracao(json_fine_tuning)
print(f"resposta {resposta}")


resposta {'lista_sugestao_resposta': [{'texto_resposta': '', 'score': 0.22037574648857117, 'lista_referencia': [[0, 0]]}, {'texto_resposta': 'Onde estava localizada a Continental Edison Company?', 'score': 0.02530529536306858, 'lista_referencia': [[0, 51]]}, {'texto_resposta': 'localizada a Continental Edison Company?', 'score': 0.01168345008045435, 'lista_referencia': [[12, 51]]}]}


In [98]:
from source.calculation.metric import qa_metric
# exact_match_score, f1_score, metric_max_over_ground_truths


In [177]:
# https://stackoverflow.com/questions/5364050/reloading-submodules-in-ipython 
# import importlib
# importlib.reload(qa_metric)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


nova importação
def


<module 'source.calculation.metric.qa_metric' from '/home/borela/fontes/exqa-complearning/source/calculation/metric/qa_metric.py'>

In [101]:
qa_metric.f1_score (prediction=resposta['lista_sugestao_resposta'][1]['texto_resposta'],ground_truth=a[0])

0

In [104]:
qa_metric.exact_match_score(prediction=resposta['lista_sugestao_resposta'][1]['texto_resposta'],ground_truth=a[0])

nova importação
def


0

In [55]:
resposta['lista_sugestao_resposta'][1]['texto_resposta']

'Onde estava localizada a Continental Edison Company?'

In [109]:
qa_metric.metric_score_over_ground_truths(metric_fn= qa_metric.f1_score, prediction='france',ground_truths=a)

[1.0, 0.6666666666666666, 1.0]

In [106]:
qa_metric.metric_max_over_ground_truths(metric_fn= qa_metric.f1_score, prediction='france',ground_truths=a)

1.0

In [159]:
path_dataset_file = "data/dataset/squad/dev-v1.1.json"

expected_version= '1.1'

In [160]:
with open(path_dataset_file) as dataset_file:
    dataset_json = json.load(dataset_file)
    if (dataset_json['version'] != expected_version):
        print('Evaluation expects v-' + expected_version +
                ', but got dataset with v-' + dataset_json['version'],
                file=sys.stderr)
    dataset = dataset_json['data']

In [161]:
path_dataset_file_pt = "data/dataset/squad/dev-v1.1-pt.json"

expected_version= '1.1'

In [162]:
with open(path_dataset_file_pt) as dataset_file:
    dataset_json_pt = json.load(dataset_file)
    if (dataset_json_pt['version'] != expected_version):
        print('Evaluation expects v-' + expected_version +
                ', but got dataset with v-' + dataset_json_pt['version'],
                file=sys.stderr)
    dataset_pt = dataset_json_pt['data']


In [164]:
dataset_json.keys(),  dataset_json_pt.keys()

(dict_keys(['data', 'version']), dict_keys(['data', 'version']))

In [165]:
len(dataset),  len(dataset_pt)


(48, 48)

In [166]:
dataset[0].keys(),dataset_pt[0].keys()

(dict_keys(['title', 'paragraphs']), dict_keys(['title', 'paragraphs']))

In [167]:
dataset[0]['paragraphs'][1].keys(), dataset_pt[0]['paragraphs'][1].keys()

(dict_keys(['context', 'qas']),
 dict_keys(['context', 'qas', 'context_original']))

In [170]:
dataset[0]['paragraphs'][1]['qas'][0].keys(), dataset_pt[0]['paragraphs'][1]['qas'][0].keys()

(dict_keys(['answers', 'question', 'id']),
 dict_keys(['answers', 'question', 'id', 'question_original']))

In [172]:
dataset[0]['paragraphs'][1]['qas'][0]['answers'][0].keys(), dataset_pt[0]['paragraphs'][1]['qas'][0]['answers'][0].keys()

(dict_keys(['answer_start', 'text']),
 dict_keys(['answer_start', 'text', 'answer_start_original', 'text_original']))

In [174]:
print(f"Dataset português")
print(f"Qtd artigos: {len(dataset_pt)}")
for artigo in dataset_pt:
    print(f"artigo.keys(): {artigo.keys()} Exemplo 1o artigo - Title: {artigo['title']}, #paragrafos {len(artigo['paragraphs'])} ")
    for paragrafo in artigo['paragraphs']:
        print(f"paragrafo.keys(): {paragrafo.keys()} Exemplo 1o paragrafo: qas: {len(paragrafo['qas'])} {paragrafo['context']} \ncontext_original {paragrafo['context_original']}")
        for pergunta_respostas in paragrafo['qas']:
            print(f"pergunta_respostas.keys(): {pergunta_respostas.keys()} Exemplo 1a pergunta_respostas: qtd respostas: {len(pergunta_respostas['answers'])}; {pergunta_respostas['question']} \nquestion_original {pergunta_respostas['question_original']}")
            for resposta in pergunta_respostas['answers']:
                print(f"resposta.keys(): {resposta.keys()} Exemplo 1a resposta {resposta}")
                break
            ground_truths = list(map(lambda x: x['text'], pergunta_respostas['answers']))
            print(f"Exemplo 1o ground_truths {ground_truths}  original: {list(map(lambda x: x['text_original'], pergunta_respostas['answers']))}")
            break
        break
    break

Dataset inglês
Qtd artigos: 48
artigo.keys(): dict_keys(['title', 'paragraphs']) Exemplo 1o artigo - Title: Super_Bowl_50, #paragrafos 203 
paragrafo.keys(): dict_keys(['context', 'qas', 'context_original']) Exemplo 1o paragrafo: qas: 26 O Super Bowl 50 foi um jogo de futebol americano para determinar o campeão da Liga Nacional de Futebol Americano (NFL) para a temporada de 2015. O campeão da Conferência Americana de Futebol Americano (AFC), Denver Broncos, derrotou o campeão da Conferência Nacional de Futebol Americano (NFC), Carolina Panthers 24–10, para ganhar seu terceiro título do Super Bowl. O jogo foi disputado em 7 de fevereiro de 2016, no Levi's Stadium na área da Baía de São Francisco em Santa Clara, Califórnia. Como este foi o 50º Super Bowl, a liga enfatizou o "aniversário de ouro" com várias iniciativas com temas de ouro, além de suspender temporariamente a tradição de nomear cada jogo do Super Bowl com números romanos (sob os quais o jogo seria conhecido como " Super Bowl

In [175]:
print(f"Dataset inglês")
print(f"Qtd artigos: {len(dataset)}")
for artigo in dataset:
    print(f"artigo.keys(): {artigo.keys()} Exemplo 1o artigo - Title: {artigo['title']}, #paragrafos {len(artigo['paragraphs'])} ")
    for paragrafo in artigo['paragraphs']:
        print(f"paragrafo.keys(): {paragrafo.keys()} Exemplo 1o paragrafo: qas: {len(paragrafo['qas'])} {paragrafo['context']}")
        for pergunta_respostas in paragrafo['qas']:
            print(f"pergunta_respostas.keys(): {pergunta_respostas.keys()} Exemplo 1a pergunta_respostas: qtd respostas: {len(pergunta_respostas['answers'])}; {pergunta_respostas['question']}")
            for resposta in pergunta_respostas['answers']:
                print(f"resposta.keys(): {resposta.keys()} Exemplo 1a resposta {resposta}")
                break
            ground_truths = list(map(lambda x: x['text'], pergunta_respostas['answers']))
            print(f"Exemplo 1o ground_truths {ground_truths}")
            break
        break
    break

Dataset inglês
Qtd artigos: 48
artigo.keys(): dict_keys(['title', 'paragraphs']) Exemplo 1o artigo - Title: Super_Bowl_50, #paragrafos 54 
paragrafo.keys(): dict_keys(['context', 'qas']) Exemplo 1o paragrafo: qas: 30 Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.
pergunt

In [ ]:
print(f"Validando se respostas estão no texto")
qtd_erro = qtd_erro_pt = qtd_pergunta = qtd_pergunta_repetida = 0
perguntas_unicas = set()
for artigo in dataset_pt:
    # artigo['title']
    for paragrafo in artigo['paragraphs']:
        contexto = paragrafo['context']
        contexto_original = paragrafo['context_original']
        for pergunta_respostas in paragrafo['qas']:
            qtd_pergunta += 1
            if len(pergunta_respostas['answers']) != 3:
                print(f"Encontradas qtd ground_truth = {len(pergunta_respostas['answers'])} qtd_pergunta {qtd_pergunta}")            
            if pergunta_respostas['question'] not in perguntas_unicas:
               perguntas_unicas.add(pergunta_respostas['question']) 
            else:
                qtd_pergunta_repetida += 1

            for resposta in pergunta_respostas['answers']:
                if resposta['text'] not in contexto:
                    print(f"PT: Resposta não está no texto artigo {artigo['title']} id: {pergunta_respostas['id']}")
                    qtd_erro_pt += 1
                if resposta['text_original'] not in contexto_original:
                    qtd_erro += 1
                    print(f"EN: Resposta não está no texto artigo {artigo['title']} id: {pergunta_respostas['id']}")
print(f"qtd_pergunta {qtd_pergunta} qtd_pergunta_repetida {qtd_pergunta_repetida}")                                
print(f"Erros encontrados em português {qtd_erro_pt} e em inglês {qtd_erro}")

In [ ]:
def evaluate(dataset, predictions):
    f1 = exact_match = total = 0
    for article in dataset:
        for paragraph in article['paragraphs']:
            for qa in paragraph['qas']:
                total += 1
                # calcular resposta

                json_fine_tuning = copy.deepcopy(json_exemplo)
                json_fine_tuning["tamanho_max_resposta"] = 30
                json_fine_tuning['texto_pergunta'] = qa['question']
                json_fine_tuning['texto_contexto'] = c
                resposta = responder_extracao(json_fine_tuning)
                print(f"resposta {resposta}")

                ground_truths = list(map(lambda x: x['text'], qa['answers']))
                prediction = predictions[qa['id']]
                exact_match += qa_metric.metric_max_over_ground_truths(
                    qa_metric.exact_match_score, prediction, ground_truths)
                f1 += qa_metric.metric_max_over_ground_truths(
                    qa_metric.f1_score, prediction, ground_truths)

    exact_match = 100.0 * exact_match / total
    f1 = 100.0 * f1 / total

    return {'exact_match': exact_match, 'f1': f1}

In [ ]:
with open(args.prediction_file) as prediction_file:
    predictions = json.load(prediction_file)
print(json.dumps(evaluate(dataset, predictions)))


In [178]:
json_fine_tuning = copy.deepcopy(json_exemplo)
json_fine_tuning["tamanho_max_resposta"] = 30
json_fine_tuning["top_k"] = 3
f1 = exact_match = total = 0
for article in dataset_pt:
    for paragraph in article['paragraphs']:
        json_fine_tuning['texto_contexto'] = paragraph['context']
        for qa in paragraph['qas']:
            total += 1
            # calcular resposta
            json_fine_tuning['texto_pergunta'] = qa['question']
            resposta = responder_extracao(json_fine_tuning)
            # print(f"resposta {resposta}")
            resposta_predita = resposta[0]['texto_resposta']
            ground_truths = list(map(lambda x: x['text'], qa['answers']))
            exact_match += qa_metric.metric_max_over_ground_truths(
                qa_metric.exact_match_score, resposta_predita, ground_truths)
            f1 += qa_metric.metric_max_over_ground_truths(
                qa_metric.f1_score, resposta_predita, ground_truths)
        print(f"mudou paragraph. total: {total} f1: {f1/total}  exact_match: {exact_match/total}  ")        
exact_match = 100.0 * exact_match / total
f1 = 100.0 * f1 / total
# return {'exact_match': exact_match, 'f1': f1}

mudou paragraph. total: 26 f1: 0.9692307692307692  exact_match: 0.9615384615384616  
mudou paragraph. total: 50 f1: 0.884  exact_match: 0.88  
mudou paragraph. total: 72 f1: 0.8916666666666667  exact_match: 0.8888888888888888  
mudou paragraph. total: 93 f1: 0.8752688172043009  exact_match: 0.8602150537634409  
mudou paragraph. total: 104 f1: 0.8884615384615384  exact_match: 0.875  
mudou paragraph. total: 112 f1: 0.8837018140589568  exact_match: 0.8571428571428571  
mudou paragraph. total: 129 f1: 0.8850744432139779  exact_match: 0.8682170542635659  
mudou paragraph. total: 151 f1: 0.8713549879112791  exact_match: 0.8609271523178808  
mudou paragraph. total: 169 f1: 0.8620456466610311  exact_match: 0.8461538461538461  
mudou paragraph. total: 189 f1: 0.8660619803476944  exact_match: 0.8518518518518519  
mudou paragraph. total: 208 f1: 0.862591575091575  exact_match: 0.8461538461538461  
mudou paragraph. total: 222 f1: 0.8532389532389532  exact_match: 0.8378378378378378  
mudou paragra

(Caderno do fine-tuning traz seguinte resultado)[https://github.com/piegu/language-models/blob/master/question_answering_BERT_large_cased_squad_v11_pt.ipynb]: 

 ***** eval metrics *****
2021-06-18 05:32:34,020 >>   epoch            =     3.0
2021-06-18 05:32:34,020 >>   eval_exact_match = 72.6774
2021-06-18 05:32:34,020 >>   eval_f1          = 84.4315
2021-06-18 05:32:34,020 >>   eval_samples     =   10917

Dúvida: por que lendo datsets há 10570? Será que exclui cópias?


In [ ]:
print(f"exact_match {exact_match} f1 {f1}")

In [148]:
from datasets import load_dataset
dataset = load_dataset("squad_v1_pt")

ModuleNotFoundError: No module named 'datasets'

## Config ...